<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Práctica Databricks Fundamentals · Unity Catalog · PySpark</title>
    <style>
        body {
            font-family: Arial, Helvetica, sans-serif;
            line-height: 1.6;
            margin: 40px;
            color: #222;
        }
        h1, h2, h3 {
            color: #B22222;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-size: 0.95em;
        }
        pre {
            background-color: #f4f4f4;
            padding: 12px;
            border-radius: 6px;
            overflow-x: auto;
        }
        .box {
            border-left: 4px solid #B22222;
            background-color: #fafafa;
            padding: 12px;
            margin: 20px 0;
        }
        .exercise {
            border-left: 4px solid #1E90FF;
            background-color: #f0f8ff;
            padding: 12px;
            margin: 20px 0;
        }
        .warning {
            border-left: 4px solid #FF8C00;
            background-color: #fff5e6;
            padding: 12px;
            margin: 20px 0;
        }
    </style>
</head>

<body>

<h1>Repaso Databricks Fundamentals · Unity Catalog · PySpark</h1>

<p>
Esta práctica forma parte de una sesión de máster orientada a consolidar los conceptos
fundamentales de <b>Databricks</b>, <b>Unity Catalog</b> y <b>PySpark</b> mediante ejecución real
de código en un notebook.
</p>

---

<h2>🧱 Contexto: Arquitectura Databricks (Repaso)</h2>

<p>Antes de empezar, recuerda los principales componentes:</p>

<ul>
    <li><b>Workspace</b>: entorno colaborativo (notebooks, jobs, repos)</li>
    <li><b>Cluster</b>: recursos de cómputo donde se ejecuta Spark</li>
    <li><b>DBFS / External Storage</b>: capa de almacenamiento</li>
    <li><b>Delta Lake</b>: formato transaccional sobre Parquet</li>
    <li><b>Unity Catalog</b>: metastore centralizado de gobierno</li>
</ul>

---

<h2>📁 Dataset</h2>

<p>Trabajaremos con un archivo CSV de ventas con la siguiente estructura:</p>

<pre>
order_id | product | quantity | price | country
</pre>

El archivo se encuentra en este github: [Enlace](https://github.com/psanzceste/databricks_master/4_repaso/data.git)

Cuando te lo descargues, ve a Unity Catalog, abre un catalogo, abre un schema, abre un volumen y subeló ahí

---

<h2>📥 Paso 1 – Carga de datos con PySpark</h2>

<p>
Usaremos la API de lectura de Spark para cargar el CSV como un DataFrame distribuido. Quiero que primero se examine la información del archivo y luego se decida sobre los diferentes parámetros de la función
</p>

<pre>
spark.read
    .option("header", ...)
    .option("inferSchema", ...)
    .csv(...)
</pre>

<p>
Una vez cargado, utilizaremos <code>display()</code> para explorar visualmente los datos.
</p>

---

<div class="exercise">
<h3>✍️ Ejercicio 1 – Exploración básica</h3>
<ul>
    <li>Examinar el contenido de CSV</li>
    <li>Cargar correctamente el DataFrame</li>
    <li>Muestra el esquema del DataFrame</li>
    <li>Cuenta el número total de registros</li>
    <li>Identifica los tipos de datos detectados por Spark</li>
</ul>
</div>

---

<h2>🔧 Paso 2 – Transformaciones con PySpark</h2>

<p>
Aplicaremos una transformación sencilla para ilustrar el modelo inmutable de Spark:
crear una nueva columna calculada.
</p>

<p>
Ejemplo conceptual:
</p>

<prep>
total_amount = quantity * price
</pre>
<pre>
df_transformed = (
    df
    .withColumn(...., col(...) * col(...))
)
</pre>

<p>
Ve al notebook, pega el código y muestra el resultado
</p>

<div class="box">
<b>Concepto clave:</b> las transformaciones en Spark son <i>lazy</i>. No se ejecutan
hasta que se lanza una acción.
</div>

---

<div class="exercise">
<h3>✍️ Ejercicio 2 – Transformaciones</h3>
<ul>
    <li>Crea una columna <code>total_amount</code></li>
    <li>Filtra las ventas con importe mayor a 100</li>
    <li>Agrupa por país y calcula el total de ventas</li>
</ul>
</div>

---

<h2>🧠 Paso 3 – Introducción a Unity Catalog</h2>

<p>
Unity Catalog proporciona un modelo jerárquico de gobierno:
</p>

<pre>
Metastore
 └── Catalog
     └── Schema
         └── Table / View / Function
</pre>

<p>
Cada nivel puede tener permisos explícitos, desacoplando seguridad y almacenamiento.
</p>

<div class="box">
<b>Importante:</b> Unity Catalog gobierna el acceso a los datos, no su procesamiento.
</div>

<p>
Analiza la estructura de tu Unity Catalog para ver si tiene sentido crear una nueva. La idea es que la jerarquía creada tenga sentido para el resto de las clases. Si crees que lo correcto sería crear algún objeto, este es el momento de crear un nuevo Catalogo o un nuevo Schema.

Si quieres crear uno nuevo, ve al Notebook, ve al Paso 3, configura los nombres que quieras darle a cada cosa y ejecuta
</p>

---

<h2>💾 Paso 4 – Crear una tabla Delta gestionada</h2>

<p>
Persistiremos el DataFrame transformado como una tabla gobernada.
</p>

<p>
Esto implica:
</p>

<ul>
    <li>Formato Delta Lake</li>
    <li>Metadatos registrados en Unity Catalog</li>
    <li>Ubicación gestionada por Databricks</li>
</ul>

Para ello ejecuta en el Notebook este comando modificando las cosas que haya que hacer

<pre>
...write.mode("overwrite").saveAsTable(...)
</pre>

---

<h3>🔍 Spark SQL vs PySpark</h3>

<p>
Una vez creada la tabla, podrás consultarla tanto con PySpark

<pre>
df = spark.table("master.repaso.sales_table")
</pre>

Como con SQL:
</p>

<pre>
SELECT * FROM tu_catalogo.tu_schema.sales_table;
</pre>

<p>
Ambos enfoques comparten el mismo motor de ejecución. Compruebalo en el Notebook
</p>

---

<div class="exercise">
<h3>✍️ Ejercicio 3 – Gobierno de datos</h3>
<ul>
    <li>Guarda el DataFrame como tabla</li>
    <li>Consulta los permisos asociados a la tabla. Para ello ve a Unity Catalog y mira en las propiedades de la tabla</li>
</ul>
</div>

---

<h2>✅ Conclusiones</h2>

<ul>
    <li>Databricks unifica ingestión, procesamiento y gobierno</li>
    <li>PySpark es la API principal para transformaciones distribuidas</li>
    <li>Unity Catalog centraliza seguridad y metadatos</li>
    <li>Delta Lake es el estándar para persistencia</li>
</ul>

<div class="box">
<b>Mensaje final:</b> Databricks no es solo Spark en la nube, es una plataforma
de datos gobernada de extremo a extremo.
</div>

</body>
</html>


Para consultar la tabla con PySpark, puedes usar el siguiente código:

python
df = spark.table("master.repaso.sales_table")
display(df)